Initialization 

Just importing necessary packages and connecting to the database.

In [2]:
from telnetlib import STATUS
import psycopg2
import pandas as pd
import sys
from tqdm import tqdm

password = 'PAiac14-'
host = 'greenplum01.corral.tacc.utexas.edu'
port = 5432
database = 'uthealth'

username = 'piacobelli'
connection = psycopg2.connect(
    host = host, 
    database = database, 
    user = username, 
    port = port, 
    password = password, 
    keepalives = 1, keepalives_idle = 100
)
connection.autocommit = True


C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [12]:
tableCreateQuery = ''' 
DROP TABLE IF EXISTS qa_reporting.dw_iqva_mbr_enrl_monthly_updated;
CREATE TABLE qa_reporting.dw_iqva_mbr_enrl_monthly_updated
(
    data_source TEXT,
    calendar_year INT,
    table_src TEXT,
    dw_row_count INT,
    src_row_count INT,
    row_count_diff INT,
    row_count_diff_percentage FLOAT,
    dw_uth_mbr_id_count INT,
    dw_src_mbr_id_count INT,
    src_mbr_count INT,
    mbr_count_diff INT,
    mbr_count_percentage FLOAT,
    date_generated DATE
);
'''

with connection.cursor() as cursor:
    cursor.execute(tableCreateQuery)

In [13]:
with connection.cursor() as cursor:
    tableInsertQuery = '''
        INSERT INTO qa_reporting.dw_iqva_mbr_enrl_monthly
        (data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, date_generated)
        SELECT data_source, 
                year, 
                table_id_src, 
                count(*),
                count(distinct uth_member_id),
                count(distinct member_id_src),
                current_date
        FROM dw_staging.iqva_member_enrollment_monthly
        GROUP BY 1,2,3;
    '''

    cursor.execute(tableInsertQuery)

      

In [14]:
with connection.cursor() as cursor:
      tableUpdateQuery = '''
            UPDATE qa_reporting.dw_iqva_mbr_enrl_monthly a
            SET src_row_count = b.row_count,
                row_count_diff = a.dw_row_count - b.row_count,
                row_count_diff_percentage = 100. * abs(a.dw_row_count - b.row_count) / b.row_count,
                src_mbr_count = b.pat_count,
                mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
                mbr_count_percentage = 100. * abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
            FROM qa_reporting.iqvia_counts b
            WHERE data_source = 'iqva' AND a.calendar_year = b.year AND a.table_src ||'2' = b.table_name;
      '''

      cursor.execute(tableUpdateQuery)

Checking to see if there are any years where the counts do not match with the raw tables.

In [15]:
countCheckQuery = '''
    SELECT * 
    FROM qa_reporting.dw_iqva_mbr_enrl_monthly
ORDER BY calendar_year;
'''
memberMonthlyDf = pd.read_sql(countCheckQuery, con=connection)
memberMonthlyDf

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_25676\610240327.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  memberMonthlyDf = pd.read_sql(countCheckQuery, con=connection)


,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
0,iqva,2006,enroll,284739058,284739058,0,0.0,30675914,30675914,30675914,0,0.0,2024-01-18
1,iqva,2006,enroll,284739058,284739058,0,0.0,30675914,30675914,30675914,0,0.0,2024-04-05
2,iqva,2006,enroll,284739058,284739058,0,0.0,30675914,30675914,30675914,0,0.0,2024-04-05
3,iqva,2007,enroll,346468836,346468836,0,0.0,34820782,34820782,34820782,0,0.0,2024-04-05
4,iqva,2007,enroll,346468836,346468836,0,0.0,34820782,34820782,34820782,0,0.0,2024-01-18
5,iqva,2007,enroll,346468836,346468836,0,0.0,34820782,34820782,34820782,0,0.0,2024-04-05
6,iqva,2008,enroll,373065024,373065024,0,0.0,37240842,37240842,37240842,0,0.0,2024-01-18
7,iqva,2008,enroll,373065024,373065024,0,0.0,37240842,37240842,37240842,0,0.0,2024-04-05
8,iqva,2008,enroll,373065024,373065024,0,0.0,37240842,37240842,37240842,0,0.0,2024-04-05
9,iqva,2009,enroll,359279585,359279585,0,0.0,35208450,35208450,35208450,0,0.0,2024-04-05


In [16]:
memberMonthlyDf.to_clipboard(excel=True, index=False)
memberMonthlyDf[(memberMonthlyDf['row_count_diff_percentage'] > 1.) | (memberMonthlyDf['mbr_count_percentage'] > 1.)]

,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated


No rows in memberMonthlyDf means that all of the rows from the raw tables are in this enrollment table at a monthly level.

Gender Count

Checking to ensure that gender has been correctly added.

In [3]:
genderCountQuery = '''
    WITH iqva_gen_cd as (
        SELECT 
            year, 
            a.pat_id, 
            CASE 
                WHEN b.pat_id IS NULL THEN 'U'
                ELSE b.der_sex
            END AS der_sex
        FROM iqvia.enroll2 a
        LEFT JOIN iqvia.enroll_synth b ON a.pat_id = b.pat_id
    ),
    iqva_gen as (
        SELECT 
            year, 
            der_sex, 
            COUNT(*) gender_count
        FROM iqva_gen_cd
        GROUP BY 1,2
    ), dw_gen AS (
        SELECT 
            year, 
            gender_cd, 
            COUNT(*) gender_count
        FROM dw_staging.iqva_member_enrollment_monthly
        GROUP BY 1,2
    )
    SELECT 
        a.year, 
        a.gender_cd, 
        a.gender_count AS dw_gender_count, 
        b.gender_count AS src_gender_count, 
        a.gender_count - b.gender_count AS gender_count_diff, 
        100. * abs(a.gender_count - b.gender_count) / b.gender_count AS gender_count_diff_percentage
    FROM iqva_gen b
    FULL OUTER JOIN dw_gen a ON a.year = b.year AND a.gender_cd = b.der_sex;
'''
 
genderCountDf = pd.read_sql(genderCountQuery,  con=connection)
genderCountDf.sort_values(['year', 'gender_cd'])

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_928\3200608546.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  genderCountDf = pd.read_sql(genderCountQuery,  con=connection)


,year,gender_cd,dw_gender_count,src_gender_count,gender_count_diff,gender_count_diff_percentage
4,2006,F,11761338,11761338,0,0.0
27,2006,M,10573018,10573018,0,0.0
39,2006,U,262404702,262404702,0,0.0
2,2007,F,12770861,12770861,0,0.0
51,2007,M,11546113,11546113,0,0.0
6,2007,U,322151862,322151862,0,0.0
13,2008,F,13459267,13459267,0,0.0
1,2008,M,12199036,12199036,0,0.0
5,2008,U,347406721,347406721,0,0.0
25,2009,F,13879840,13879840,0,0.0


Plan Type Count

In [18]:
planTypeQuery = '''
    WITH iqva_enroll AS (
        SELECT 
            year, 
            pat_id, 
            plan_type
        FROM iqvia.enroll2 a
        LEFT JOIN reference_tables.ref_plan_type c ON c.data_source  = 'iqva' AND c.plan_type_src = a.prd_type
    ),
    iqva_plans AS (          
        SELECT 
            year, 
            CASE WHEN plan_type IS NULL THEN 'U' ELSE plan_type END AS plan_type, 
            COUNT(*) plan_count
        FROM iqva_enroll a
        GROUP BY 1,2
    ),
    dw_plans AS (
        SELECT 
            year, 
            CASE WHEN plan_type IS NULL THEN 'U' ELSE plan_type END AS plan_type,
            COUNT(*) plan_count
        FROM dw_staging.iqva_member_enrollment_monthly
        GROUP BY 1,2
    )
    SELECT 
        a.year, 
        a.plan_type, 
        a.plan_count AS dw_plan_count, 
        b.plan_count AS src_plan_count, 
        a.plan_count - b.plan_count AS plan_count_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count AS plan_count_diff_percentage
    FROM iqva_plans b
    FULL OUTER JOIN dw_plans a ON a.year = b.year AND a.plan_type = b.plan_type
    order by year;
'''

planCountDf = pd.read_sql(planTypeQuery,  con=connection)
planCountDf.sort_values(['year', 'plan_type'])

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_25676\640206686.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  planCountDf = pd.read_sql(planTypeQuery,  con=connection)


,year,plan_type,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage
1,2006,CDHP,715632,715632,0,0.0
4,2006,FFS,15492469,15492469,0,0.0
0,2006,HMO,81070648,81070648,0,0.0
3,2006,POS,24654013,24654013,0,0.0
6,2006,PPO,160385054,160385054,0,0.0
...,...,...,...,...,...,...
109,2022,UNK,2178159,2178159,0,0.0
116,2023,CDHP,3443250,3443250,0,0.0
114,2023,HMO,6939852,6939852,0,0.0
117,2023,PPO,12651689,12651689,0,0.0


In [19]:
planCountDf[planCountDf['plan_count_diff_percentage'] > 1.0]

,year,plan_type,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage


In [25]:
yearlyPlanCountDf = planCountDf.groupby('year')[['dw_plan_count', 'src_plan_count']].sum()
yearlyPlanCountDf['plan_count_diff'] = yearlyPlanCountDf['dw_plan_count'] - yearlyPlanCountDf['src_plan_count']
yearlyPlanCountDf['plan_count_diff_percentage'] = 100.* abs(yearlyPlanCountDf['plan_count_diff'] / yearlyPlanCountDf['src_plan_count'])
yearlyPlanCountDf

,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage
year,,,,
2006,284739058,284739058,0,0.0
2007,346468836,346468836,0,0.0
2008,373065024,373065024,0,0.0
2009,359279585,359279585,0,0.0
2010,312564976,312564976,0,0.0
2011,304518538,304518538,0,0.0
2012,269545623,269545623,0,0.0
2013,247873630,247873630,0,0.0
2014,258722696,258722696,0,0.0
